In [623]:
import json
import datetime
import dateutil
from types import SimpleNamespace

import faker
import pandas as pd
from sqlalchemy import create_engine, URL
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import sql

DB_CONFIG_FILE = "config.json"  # Ignored by git!
INSERT_DRY_RUN = False


TODO extract magic numbers
TODO generate remaining data

In [624]:
with open(DB_CONFIG_FILE, mode='r') as fp:
    DB_CONFIG = json.load(fp, object_hook=lambda x: SimpleNamespace(**x))

db_url = URL.create(
    drivername="mysql",
    username=DB_CONFIG.db_username,
    password=DB_CONFIG.db_password,
    host=DB_CONFIG.db_host,
    database=DB_CONFIG.db_database,
    port=DB_CONFIG.db_port
)
engine = create_engine(db_url)
session = Session(engine)

Base = automap_base()
Base.prepare(autoload_with=engine)

queries = {name: session.query(table).filter(sql.false()) for name, table in Base.metadata.tables.items()}
dfs = {name: pd.read_sql(query.statement, engine) for name, query in queries.items()}

In [625]:
def append_to_df(name: str, new_lines: pd.DataFrame) -> None:
    # RIP SOLID
    print(f"Generating \"{name}\" [{len(new_lines.index)}] done")
    assert str(new_lines.columns) == str(dfs[name].columns)
    dfs[name] = pd.concat([dfs[name], new_lines])

In [626]:
fake = faker.Faker('pl_PL')

employee positions

In [627]:
# employee_positions = pd.DataFrame([
#     [0, 'mechanic', 'maintains vehicles'],
#     [1, 'admin', 'keeps the system running']
# ], columns=['id', 'position_name', 'description'])
# append_to_df('employee_position', employee_positions)

employees

In [628]:
# employees = pd.DataFrame(
#     [(i, fake.first_name(), fake.last_name(), 0) for i in range(17)] + [(i, fake.first_name(), fake.last_name(), 1)
#                                                                         for i in range(17, 20, 1)],
#     columns=['id', 'name', 'surname', 'employee_position_id']
# )
# append_to_df('employee', employees)

customers

In [629]:
# customers = pd.DataFrame(
#     [(i, fake.first_name(), fake.last_name()) for i in range(3000)],
#     columns=['id', 'name', 'surname']
# )
# append_to_df('customer', customers)

driving licences

In [630]:
# fake_drv_lic_number = lambda: f"{fake.random.randint(0, 9999):04}/{fake.random.randint(0, 99):02}/{fake.random.randint(0, 9999):04}"
# fake_drv_lic = lambda category, number: pd.DataFrame(
#     [(customer_id,
#       fake_drv_lic_number(),
#       category,
#       start_d.strftime("%d.%m.%Y"),
#       (start_d + dateutil.relativedelta.relativedelta(years=15)).strftime("%d.%m.%Y"))
#      for customer_id, start_d in zip(
#         customers.sample(number)['id'].sort_values(),
#         [fake.date_between(datetime.date(2008, 2, 21), datetime.date(2023, 5, 16)) for _ in range(number)])],
#     columns=['customer_id', 'drivers_license_number', 'drivers_license_category', 'valid_from', 'valid_until']
# )
# driving_licences = pd.concat([
#     fake_drv_lic('B', int(3000 * 0.98)),
#     pd.concat([fake_drv_lic(cat, int(3000 * 0.03)) for cat in ('A', 'BE', 'C')]),
#     pd.concat([fake_drv_lic(cat, int(3000 * 0.001)) for cat in
#                ('M', 'B1', 'C1', 'D1', 'D', 'C1E', 'CE', 'D1E', 'DE', 'T', 'F')])
# ])
# append_to_df('driving_licence', driving_licences)

car_type

In [631]:
car_type = pd.DataFrame(['Hatchback', 'Kombi', 'Sedan', 'Liftback', 'Van', 'SUV', 'Crossover', 'Coupe'], columns=['name'])
append_to_df('car_type', car_type)

Generating "car_type" [8] done


brand

In [632]:
brand = pd.DataFrame(['Toyota', 'Volkswagen', 'Ford', 'Honda', 'Nissan', 'Hyundai', 'Chevrolet', 'Kia',
'Mercedes', 'BMW', 'Fiat', 'Opel', 'Peugeot', 'Citroen', 'Audi', 'Skoda', 'Volvo', 'Mazda', 'Seat',
'Suzuki', 'Mitsubishi', 'Land Rover', 'Jeep', 'Porsche', 'Alfa Romeo', 'Chrysler', 'Jaguar', 'Ferrari', 'Infiniti', 
'Lexus', 'Dacia', 'Mini', 'Smart', 'Renault'], columns=['name'])
append_to_df('brand', brand)

Generating "brand" [34] done


parameter

In [633]:
parameter = pd.DataFrame([
    [0, 'color', 'color', 's'],
    [1, 'drive_type', 'drive type', 's'],
    [2, 'engine_capacity', 'engine capacity', 'f'],
    [3, 'engine_power', 'engine_power', 'i'],
    [4, 'fuel_type', 'fuel type', 's'],
    [5, 'gearbox_type', 'gearbox type', 's'],
    [6, 'mileage', 'mileage', 'i'],
    [7, 'seat_number', 'seat number', 'i']], 
    columns=['id', 'name', 'description', 'type'])
append_to_df('parameter', parameter)

Generating "parameter" [8] done


model

In [634]:
model = pd.DataFrame([
    [0, 'Astra', 'B', 'Opel', 'Hatchback'],
    [1, '3', 'B', 'Mazda', 'Sedan'],
    [2, 'A4', 'B', 'Audi', 'Sedan'],
    [3, 'A6', 'B', 'Audi', 'Sedan'],
    [4, 'Punto', 'B', 'Fiat', 'Hatchback'],
    [5, 'Civic', 'B', 'Honda', 'Hatchback'],
    [6, 'Focus', 'B', 'Ford', 'Hatchback'],
    [7, 'Golf', 'B', 'Volkswagen', 'Hatchback'],
    [8, 'Passat', 'B', 'Volkswagen', 'Sedan'],
    [9, 'Clio', 'B', 'Renault', 'Hatchback'],
    [10, 'Megane', 'B', 'Renault', 'Hatchback'],
    [11, 'Corolla', 'B', 'Toyota', 'Hatchback'],
    [12, 'Yaris', 'B', 'Toyota', 'Hatchback'],
    [13, 'Auris', 'B', 'Toyota', 'Hatchback'],
    [14, 'Avensis', 'B', 'Toyota', 'Sedan'],
    [15, 'Ceed', 'B', 'Kia', 'Hatchback'],
    [16, 'Rio', 'B', 'Kia', 'Hatchback'],
    [17, 'S40', 'B', 'Volvo', 'Sedan'],
    [18, 'V40', 'B', 'Volvo', 'Hatchback'],
    [19, 'V50', 'B', 'Volvo', 'Hatchback'],
    [20, 'XC60', 'B', 'Volvo', 'SUV'],
    [21, 'XC70', 'B', 'Volvo', 'SUV'],
    [22, 'C4', 'B', 'Citroen', 'Hatchback'],
    [23, 'C5', 'B', 'Citroen', 'Sedan'],
    [24, 'C6', 'B', 'Citroen', 'Sedan'],
    [25, 'Qashqai', 'B', 'Nissan', 'SUV'],
    [26, 'Juke', 'B', 'Nissan', 'SUV'],
    [27, 'Micra', 'B', 'Nissan', 'Hatchback'],
    [28, 'Note', 'B', 'Nissan', 'Hatchback']],
    columns=['id', 'name', 'licence_type_required', 'car_brand_name', 'car_type_name']
)
append_to_df('model', model)

Generating "model" [29] done


model_parameter

In [635]:
model_parameter = pd.DataFrame(
    [[i, 'red', None, i, 0] for i in range (0, 29)] +
    [[i+33, 'manual', None, i, 1] for i in range(29)] +
    [[i+66, None, 5, i, 7] for i in range (29)],
    columns=['id', 'text_value', 'numerical_value', 'model_id', 'parameter_id']
)

append_to_df('model_parameter', model_parameter)

Generating "model_parameter" [87] done


In [636]:
[[i, 'red', None, i, 0] for i in range(2)]

[[0, 'red', None, 0, 0], [1, 'red', None, 1, 0]]

In [637]:
# ['brand', 'car', 'model', 'car_type', 'customer', 'driving_licence', 'employee', 'employee_position', 'insurance', 'invoice', 'model_parameter', 'parameter', 'registration_certificate', 'rental_order', 'technical_inspection'])
adding_queue = ['parameter', 'brand', 'car_type', 'model', 'model_parameter']

## Execute

In [638]:
print(" GENERATE DATA ".center(60, '='))
print(" INSERT ".center(60, '='))

for name in adding_queue:
    df = dfs[name]
    aff_rows = df.to_sql(name=name,
                         con=engine,
                         if_exists='append',
                         index=False,
                         method='multi' if not INSERT_DRY_RUN else lambda pd_table, conn, keys, data_iter: len(
                             list(data_iter)))
    print(f"{'[DRY RUN] ' if INSERT_DRY_RUN else ''}INSERT to \"{name}\" affected {aff_rows} rows")

print(" END ".center(60, '='))

====================== GENERATE DATA =======================
========================== INSERT ==========================
INSERT to "parameter" affected 8 rows
INSERT to "brand" affected 34 rows
INSERT to "car_type" affected 8 rows
INSERT to "model" affected 29 rows
INSERT to "model_parameter" affected 87 rows
=========================== END ============================
